# **Task 1: LLM Fundamentals & Generation Analysis**

## **Overview**
This task provides a **hands-on introduction** to the fundamentals of large language models (LLMs) using the **Llama 3-8B Instruct** model.  
It walks through model loading, text generation experiments, and key performance analyses, focusing on **latency**, **diversity**, and **memory usage**.

---

## **Step 1: Environment Setup (Cell 2)**

- **Objective:**  
  Create a clean and reproducible environment for running all experiments.

- **Actions:**
  - **Import Libraries:**  
    Load all required libraries for model handling, generation, and visualization.  
  - **Configure Environment:**  
    Define a `set_seed` function to ensure reproducibility by fixing random seeds.  
    Verify that a **GPU** is available for efficient computation.

---

## **Step 2: Model Loading (Cell 4)**

- **Objective:**  
  Load the target **Llama 3-8B Instruct** model and prepare it for inference.

- **Actions:**
  - **Select Model:**  
    Set `MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"`.  
  - **Load with Fallback:**  
    Use a `try...except` block to attempt loading the main model.  
    If it fails (e.g., due to limited memory), load a smaller fallback model (`FALLBACK_MODEL_ID`) to ensure the notebook remains executable.  
  - **Model Summary:**  
    Print key information about the loaded model, including size and configuration.

---

## **Step 3: Experiment 1 — Effect of Temperature on Diversity (Cell 5)**

- **Objective:**  
  Understand how the **temperature** hyperparameter influences the **randomness and diversity** of generated text.

- **Actions:**
  - **Set Parameters:**  
    Define a fixed input `PROMPT` and a list of `TEMPERATURES` to test.  
  - **Generate Samples:**  
    Loop through each temperature value and generate multiple text samples using  
    `model.generate(..., do_sample=True)` so that temperature actually affects randomness.  
  - **Record Results:**  
    Save all generated outputs for side-by-side comparison and qualitative analysis.

---

## **Step 4: Experiment 2 — Input Length vs. Prefill Latency (Cell 6)**

- **Objective:**  
  Demonstrate that **longer input sequences** lead to **higher prefill latency** before generation begins.

- **Actions:**
  - **Define Input Sizes:**  
    Create a list of token lengths (`INPUT_LENGTHS`) to test various input sizes.  
  - **Precise Timing:**  
    Use `torch.cuda.Event` for high-precision GPU time measurement.  
  - **Measure Prefill Stage:**  
    For each input length, perform warm-up runs, then measure time for generating one token (`max_new_tokens=1`) to isolate prefill cost.  
  - **Visualize:**  
    Plot a line graph showing **input length (x-axis)** vs. **average latency (y-axis)**.

---

## **Step 5: Experiment 3 — Real-Time Memory Tracking During Generation (Cell 8)**

- **Objective:**  
  Track and visualize **GPU memory usage** step-by-step as tokens are generated, illustrating the growth of the **KV Cache**.

- **Actions:**
  - **Design a “Memory Hook”:**  
    Implement a custom class `MemoryUsageCallback` inheriting from `transformers.StoppingCriteria`.  
    Its `__call__` method records current GPU memory after each token generation, returning `False` to continue.  
  - **Run Monitored Generation:**  
    After GPU warm-up, create an instance of the callback and pass it to `model.generate()` via `stopping_criteria`.  
  - **Analyze and Plot:**  
    Convert recorded memory data into a DataFrame and plot the **memory usage curve** over generation steps.

---

## **Step 6: Verifying the Space Complexity of KV Cache (Cell 9)**

- **Objective:**  
  Quantitatively confirm that memory usage grows **linearly** with sequence length — proving **O(L)** space complexity for the KV Cache.

- **Actions:**
  - **Linear Regression:**  
    Apply `scipy.stats.linregress` to fit a linear model between the number of generated tokens and memory increase.  
  - **Interpret Results:**  
    - The **slope** indicates average memory growth (in MB) per generated token.  
    - The **R² value** close to 1 (e.g., > 0.99) validates a strong linear correlation, confirming the **O(L)** relationship.

---

## **Step 7: Summary and Analysis (Cell 10)**

- **Objective:**  
  Summarize all experimental findings and explain the underlying principles observed.

- **Actions:**
  - **Write Analysis:**  
    Discuss results and plots from all experiments, highlighting:  
    - The trade-off between temperature and diversity.  
    - The linear scaling of prefill latency with input length.  
    - The direct, linear memory growth due to KV caching.  
  - Synthesize these insights into a clear, data-driven conclusion about **LLM generation efficiency** and **scalability**.


In [2]:
### Cell 2: Environment Setup and Dependency Installation
import os
import random
import time
import sys
from typing import Optional

import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.stopping_criteria import StoppingCriteria
from transformers.utils import logging
import seaborn as sns
import matplotlib.pyplot as plt

RESULTS_DIR = "./results"
FIGURES_DIR = "./figures"

os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

logging.set_verbosity_warning()
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    # print GPU diagnostics here
else:
    DEVICE = torch.device("cpu")
    # print CPU-only notice

print(f"CUDA version: {torch.version.cuda}")
print(f"PyTorch version: {torch.__version__}")
print(f"Python version: {sys.version}")

def set_seed(seed: int = 42) -> None:
    """Seed Python, NumPy, and PyTorch RNGs for reproducible lab runs."""
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)

def require_gpu(task: str) -> None:
    """Raise a descriptive error if a GPU is required but not available."""
    if DEVICE.type == "cuda":
        return
    raise RuntimeError(f"Skipped: {task} requires GPU. Please set DEVICE to 'cuda'.")

set_seed()
sns.set_theme(style="darkgrid")
plt.rcParams.update({"font.size": 14})
print("Environment initialised.")

CUDA version: 12.8
PyTorch version: 2.9.1+cu128
Python version: 3.13.3 (main, Apr  9 2025, 04:03:52) [Clang 20.1.0 ]
Environment initialised.


In [4]:
# ### Cell 3: Hugging Face Login
from huggingface_hub import login, HfFolder
from getpass import getpass

# Check if a Hugging Face token is already set in the environment.
if not os.getenv("HUGGING_FACE_HUB_TOKEN"):
    try:
        # Prompt user for Hugging Face access token if not found.
        hf_token = getpass("Please enter your Hugging Face access token: ")
        login(token=hf_token, add_to_git_credential=True)
        print("   Hugging Face login successful!")
    except Exception as e:
        print(f"Login failed: {e}. Model loading may fail later.")
else:
    print("   Hugging Face token detected.")

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
   Hugging Face login successful!


In [5]:
### Cell 4: Load Model and Tokenizer
MODEL_ID = "meta-llama/Llama-3-8B-Instruct"
FALLBACK_MODEL_ID = "meta-llama/Llama-3-8B-Instruct"

model: Optional[AutoModelForCausalLM] = None
tokenizer: Optional[AutoTokenizer] = None

candidates = [MODEL_ID, FALLBACK_MODEL_ID]

for candidate in candidates:
    model = AutoModelForCausalLM.from_pretrained(candidate).to(DEVICE)
    tokenizer = AutoTokenizer.from_pretrained(candidate).to(DEVICE)
    if model is not None and tokenizer is not None:
        break

if model is None or tokenizer is None:
    raise RuntimeError("Failed to load any model/tokenizer candidates.")

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

model.eval()
print(f"Model loaded: {model.__class__.__name__} ({model.config.model_type})")
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Model device: {model.device}")
print(f"Tokenizer loaded: {tokenizer.__class__.__name__} ({tokenizer.name_or_path})")

OSError: meta-llama/Llama-3-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
### Cell 5: Experiment 1 - Effect of Temperature on Generation Diversity
print("--- Experiment 1: Temperature sweep ---")

PROMPT = "The sum of 2 and 2 is "
TEMPERATURES = [0.0, 0.5, 1.0]
NUM_SAMPLES_PER_TEMP = 5
MAX_NEW_TOKENS = 10

if model is None or tokenizer is None:
    raise RuntimeError("Model or tokenizer not loaded. Please check the previous cells.")
require_gpu("temperature")

records = []

for temp in TEMPERATURES:
    for sample_id in range(1, NUM_SAMPLES_PER_TEMP + 1):
        tokenized_prompt = tokenizer.encode(PROMPT, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            output = model.generate(tokenized_prompt, max_new_tokens=MAX_NEW_TOKENS, temperature=temp)
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
        records.append({
            "temperature": temp,
            "sample_id": sample_id,
            "output": decoded_output
        })

df_temperature = pd.DataFrame(records)
summary_temperature = df_temperature.groupby("temperature").agg({"output": "nunique"})

df_temperature.to_csv(f"{RESULTS_DIR}/task1_step2__full.csv", index=False)
summary_temperature.to_csv(f"{RESULTS_DIR}/task1_step2__summary.csv", index=False)
print("Temperature sweep complete. Summary statistics:")


In [ ]:
### Cell 6: Experiment 2 - Effect of Input Length on Prefilling Latency
print("--- Experiment 2: Input length vs. prefill latency ---")

LATENCY_INPUT_LENGTHS = [2 ** i for i in range(4, 8)]
NUM_WARMUP = 10
NUM_TRIALS = 10
MAX_NEW_TOKENS = 10

latency_records = []

for input_length in LATENCY_INPUT_LENGTHS:
    inputs = tokenizer("a" * input_length, return_tensors="pt").to(DEVICE)
    for _ in range(NUM_WARMUP):
        with torch.no_grad():
            model.generate(inputs, max_new_tokens=MAX_NEW_TOKENS)
    for _ in range(NUM_TRIALS):
        start_time = time.time()
        with torch.no_grad():
            model.generate(inputs, max_new_tokens=MAX_NEW_TOKENS)
        latency_records.append({
            "length": input_length,
            "latency": time.time() - start_time
        })

df_latency = pd.DataFrame(latency_records)

summary_latency = df_latency.groupby("length").agg({"latency": ["mean", "std"]})
plt.figure(figsize=(10, 6))
plt.plot(df_latency["length"], df_latency["latency"], marker='o', linestyle='-', color='b')
plt.xlabel("Input Length (tokens)")
plt.ylabel("Latency (seconds)")
plt.title("Latency vs. Input Length")
plt.grid(True)
plt.savefig(f"{FIGURES_DIR}/task1_step3__latency.png")
plt.show()

In [ ]:
### Cell 8: Experiment 3 - Real-time GPU Memory Usage During Generation
print("--- Experiment 3: Real-time memory trace ---")

df_memory_steps = pd.DataFrame()

if DEVICE.type != "cuda":
    print("Skipped: requires GPU trace from the previous cell.")
else:
    MAX_GENERATION_LENGTH = 10
    PROMPT = "The sum of 2 and 2 is "
    INPUT_LENGTH = tokenizer.encode(PROMPT, return_tensors="pt").shape[1]

    # TODO: prepare input_ids at the specified length
    input_ids = tokenizer.encode(PROMPT, return_tensors="pt").to(DEVICE)

    class MemoryUsageCallback(StoppingCriteria):
        def __init__(self, device):
            self.device = device
            self.memory_usage = []

        def __call__(self, input_ids, scores, **kwargs):
            self.memory_usage.append(torch.cuda.memory_allocated(self.device) / 1024**2)
            return False

        def increases_mb(self):
            return self.memory_usage

    callback = MemoryUsageCallback(DEVICE)

    # TODO: run warmup generation without callback
    for _ in range(NUM_WARMUP):
        with torch.no_grad():
            model.generate(input_ids, max_new_tokens=MAX_NEW_TOKENS)

    with torch.no_grad():
        model.generate(input_ids, max_new_tokens=MAX_NEW_TOKENS, stopping_criteria=callback)

    # TODO: convert callback outputs into df_memory_steps and save CSV
    df_memory_steps = pd.DataFrame(callback.increases_mb())
    df_memory_steps.to_csv(f"{RESULTS_DIR}/task1_step5__memory.csv", index=False)

    # TODO: plot per-token memory increase and save the figure
    fig, ax = plt.subplots()
    sns.lineplot(df_memory_steps)
    ax.set_xlabel("Generation Step")
    ax.set_ylabel("Memory Usage (MB)")
    ax.set_title("Memory Usage During Generation")
    ax.grid(True)
    plt.savefig(f"{FIGURES_DIR}/task1_step5__memory.png")
    plt.show()
    print(df_memory_steps.tail())


In [ ]:
### Cell 9: Space Complexity Verification
print("--- Space complexity verification ---")

if DEVICE.type != "cuda":
    print("Skipped: requires GPU trace from the previous cell.")
elif df_memory_steps.empty:
    print("No memory trace data available. Run Experiment 3 before this analysis.")
else:
    # TODO: run linear regression on df_memory_steps and report slope/intercept/R^2
    # slope, intercept, r_value, p_value, std_err = linregress(...)
    # print(...)
    pass


In [ ]:
### Cell 10: List all generated artifacts for Task 1
print("Task 1 complete. Generated artifacts:")

# TODO: iterate over output directories and list generated files
# if os.path.isdir(FIGURES_DIR):
#     ...
# if os.path.isdir(RESULTS_DIR):
#     ...
